In [1]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark")

cursor = conn.cursor()

In [2]:
df_stores = pd.read_csv('Stores.csv')

print(df_stores.head())

print(df_stores.info())

df_stores.columns = df_stores.columns.str.lower()
df_stores.columns = df_stores.columns.str.replace(" ","_")

df_stores['open_date'] = pd.to_datetime(df_stores['open_date'])

# df_stores['open_date'] = df_stores['open_date'].dt.strftime('%Y-%m-%d')



   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB
None


In [3]:
df_stores.columns

Index(['storekey', 'country', 'state', 'square_meters', 'open_date'], dtype='object')

In [3]:
columns = df_stores.columns
# Prefix to add
prefix = 'st.'

# Convert column names
converted_columns = [f'{prefix}{col}' for col in columns]
columns = ['c.'.join(column) for column in columns]

for i in converted_columns:
    print(i)

st.storekey
st.country
st.state
st.square_meters
st.open_date


In [4]:
df_stores['square_meters'] = df_stores['square_meters'].fillna(0.0)

In [5]:
df_stores[df_stores.duplicated()]

,storekey,country,state,square_meters,open_date


In [9]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   storekey       67 non-null     int64         
 1   country        67 non-null     object        
 2   state          67 non-null     object        
 3   square_meters  67 non-null     float64       
 4   open_date      67 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 2.7+ KB


In [6]:
# insert_stores_values = '''
#     INSERT INTO stores (storekey, 
#                         country, 
#                         state, 
#                         square_meters, 
#                         open_date)
#     VALUES (%s, %s, %s, %s, %s)
#    '''
insert_stores_values = '''
    INSERT INTO stores (storekey, country, state, square_meters, open_date)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    country=VALUES(country),
    state=VALUES(state),
    square_meters=VALUES(square_meters),
    open_date=VALUES(open_date)
'''


for index, row in df_stores.iterrows():
    cursor.execute(insert_stores_values, (
        row['storekey'],
        row['country'],
        row['state'],
        row['square_meters'],
        row['open_date']
))
conn.commit()        